In [1]:
import numpy as np
import pandas as pd
import janitor
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import glob
import json
import requests

In [2]:
# Read all the month data for 2019
path = "/Users/lee14257/Development/CMU/Perspectives_in Data_Science/Project/2019"
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

bike2019 = pd.concat(li, axis=0, ignore_index=True)

In [3]:
len(bike2019)

3398417

In [4]:
bike2019.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,429,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,Member
1,1935,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,Member
2,563,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,Member
3,625,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,Member
4,312,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,Member


In [5]:
bike2019['Start date'] = pd.to_datetime(bike2019['Start date'], utc=False)
bike2019['End date'] = pd.to_datetime(bike2019['End date'], utc=False)

In [6]:
# Number of unique stations
len(bike2019['End station number'].unique())

581

## Create Availability table

In [7]:
# Create Availability table
drop_df = bike2019[['Start station number', 'Start station', 'Start date', 'Bike number']].assign(act=-1)
drop_df.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
pick_df = bike2019[['End station number', 'End station', 'End date', 'Bike number']].assign(act=1)
pick_df.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
act_df = pd.concat([drop_df, pick_df])
act_df = act_df.assign(reshuffle=0)

In [8]:
act_df['station_id'].replace(0, 31127,inplace=True)
act_df = act_df.sort_values(['station_id', 'date'])

In [9]:
act_df

,station_id,station_name,date,bike_id,act,reshuffle
343942,31000,Eads St & 15th St S,2019-01-02 09:46:15,W23821,-1,0
343993,31000,Eads St & 15th St S,2019-01-02 09:58:13,W21229,-1,0
344460,31000,Eads St & 15th St S,2019-01-02 12:09:20,W23657,1,0
344552,31000,Eads St & 15th St S,2019-01-02 12:12:59,W23657,-1,0
346772,31000,Eads St & 15th St S,2019-01-02 17:34:18,W23821,1,0
...,...,...,...,...,...,...
2886440,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0
2886446,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0
2886696,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0
2886697,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0


In [10]:
# Testing cumsum to get availability
act_df_temp = act_df.assign(availability = act_df.groupby('station_id')['act'].transform(pd.Series.cumsum))

In [11]:
act_df_temp

,station_id,station_name,date,bike_id,act,reshuffle,availability
343942,31000,Eads St & 15th St S,2019-01-02 09:46:15,W23821,-1,0,-1
343993,31000,Eads St & 15th St S,2019-01-02 09:58:13,W21229,-1,0,-2
344460,31000,Eads St & 15th St S,2019-01-02 12:09:20,W23657,1,0,-1
344552,31000,Eads St & 15th St S,2019-01-02 12:12:59,W23657,-1,0,-2
346772,31000,Eads St & 15th St S,2019-01-02 17:34:18,W23821,1,0,-1
...,...,...,...,...,...,...,...
2886440,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0,20
2886446,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0,21
2886696,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0,20
2886697,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0,19


## Identify Reshuffled rows

In [12]:
# Add van reshuffled as additional rows
df_reshuffle = act_df.sort_values(['bike_id', 'date'])
df_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle
1363967,31124,14th & Irving St NW,2019-11-21 08:47:33,21054,-1,0
1363967,31101,14th & V St NW,2019-11-21 08:54:07,21054,1,0
1364146,31101,14th & V St NW,2019-11-21 08:55:49,21054,-1,0
1364146,31214,17th & Corcoran St NW,2019-11-21 09:02:29,21054,1,0
1364411,31214,17th & Corcoran St NW,2019-11-21 09:12:15,21054,-1,0


In [13]:
# Shift previous stations to compare and determine reshuffled rows
df_reshuffle['station_id_prev'] = df_reshuffle['station_id'].shift()
df_reshuffle['station_name_prev'] = df_reshuffle['station_name'].shift()
df_reshuffle['bike_id_prev'] = df_reshuffle['bike_id'].shift()
df_reshuffle['date_prev'] = df_reshuffle['date'].shift()

In [14]:
df_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle,station_id_prev,station_name_prev,bike_id_prev,date_prev
1363967,31124,14th & Irving St NW,2019-11-21 08:47:33,21054,-1,0,NaN,NaN,NaN,NaT
1363967,31101,14th & V St NW,2019-11-21 08:54:07,21054,1,0,31124.0,14th & Irving St NW,21054,2019-11-21 08:47:33
1364146,31101,14th & V St NW,2019-11-21 08:55:49,21054,-1,0,31101.0,14th & V St NW,21054,2019-11-21 08:54:07
1364146,31214,17th & Corcoran St NW,2019-11-21 09:02:29,21054,1,0,31101.0,14th & V St NW,21054,2019-11-21 08:55:49
1364411,31214,17th & Corcoran St NW,2019-11-21 09:12:15,21054,-1,0,31214.0,17th & Corcoran St NW,21054,2019-11-21 09:02:29


In [15]:
# Rows that need reshuffle (add rows)
need_reshuffle = df_reshuffle[(df_reshuffle['act'] == -1) & 
             (df_reshuffle['station_id'] != df_reshuffle['station_id_prev']) & 
             (df_reshuffle['bike_id'] == df_reshuffle['bike_id_prev'])]
need_reshuffle[['station_id_prev']] = need_reshuffle[['station_id_prev']].astype(int)
need_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle,station_id_prev,station_name_prev,bike_id_prev,date_prev
1365289,31124,14th & Irving St NW,2019-11-21 11:42:54,21054,-1,0,31227,13th St & New York Ave NW,21054,2019-11-21 09:21:34
1375451,31227,13th St & New York Ave NW,2019-11-22 16:42:55,21054,-1,0,31254,15th & K St NW,21054,2019-11-22 08:54:57
1388885,31302,Wisconsin Ave & Newark St NW,2019-11-25 07:55:12,21054,-1,0,31246,M St & Pennsylvania Ave NW,21054,2019-11-23 17:21:11
2761007,31299,Connecticut Ave & R St NW,2019-12-04 15:27:31,21054,-1,0,31253,19th & K St NW,21054,2019-12-03 08:12:44
2764396,31242,18th St & Pennsylvania Ave NW,2019-12-04 22:09:07,21054,-1,0,31203,14th & Rhode Island Ave NW,21054,2019-12-04 15:37:13


In [16]:
# Add mid point dates for reshuffling vans
need_reshuffle['date_1q'] = need_reshuffle['date_prev'] + ((need_reshuffle['date'] - need_reshuffle['date_prev'])//3)
need_reshuffle['date_2q'] = need_reshuffle['date_prev'] + (2*(need_reshuffle['date'] - need_reshuffle['date_prev'])//3)
need_reshuffle['date_1q'] = need_reshuffle["date_1q"].dt.strftime("%Y-%m-%d %H:%M:%S")
need_reshuffle['date_2q'] = need_reshuffle["date_2q"].dt.strftime("%Y-%m-%d %H:%M:%S")
need_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle,station_id_prev,station_name_prev,bike_id_prev,date_prev,date_1q,date_2q
1365289,31124,14th & Irving St NW,2019-11-21 11:42:54,21054,-1,0,31227,13th St & New York Ave NW,21054,2019-11-21 09:21:34,2019-11-21 10:08:40,2019-11-21 10:55:47
1375451,31227,13th St & New York Ave NW,2019-11-22 16:42:55,21054,-1,0,31254,15th & K St NW,21054,2019-11-22 08:54:57,2019-11-22 11:30:56,2019-11-22 14:06:55
1388885,31302,Wisconsin Ave & Newark St NW,2019-11-25 07:55:12,21054,-1,0,31246,M St & Pennsylvania Ave NW,21054,2019-11-23 17:21:11,2019-11-24 06:12:31,2019-11-24 19:03:51
2761007,31299,Connecticut Ave & R St NW,2019-12-04 15:27:31,21054,-1,0,31253,19th & K St NW,21054,2019-12-03 08:12:44,2019-12-03 18:37:39,2019-12-04 05:02:35
2764396,31242,18th St & Pennsylvania Ave NW,2019-12-04 22:09:07,21054,-1,0,31203,14th & Rhode Island Ave NW,21054,2019-12-04 15:37:13,2019-12-04 17:47:51,2019-12-04 19:58:29


In [17]:
# Create Reshuffle rows
reshuffle_pick = need_reshuffle[['station_id_prev', 'station_name_prev', 'date_1q', 'bike_id_prev']].assign(act=-1)
reshuffle_pick.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
reshuffle_drop = need_reshuffle[['station_id', 'station_name', 'date_2q', 'bike_id']].assign(act=1)
reshuffle_drop.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
act_df_reshuffle = pd.concat([reshuffle_pick, reshuffle_drop])
act_df_reshuffle = act_df_reshuffle.assign(reshuffle=1)

In [18]:
act_df_reshuffle = act_df_reshuffle.sort_values(['bike_id', 'date'])

## Combine original and reshuffle table together

In [19]:
# Add original and reshuffled table
act_df = act_df.reset_index(drop=True)
act_df_reshuffle = act_df_reshuffle.reset_index(drop=True)
final_act_df = pd.concat([act_df, act_df_reshuffle], ignore_index=True)
final_act_df.head()

,station_id,station_name,date,bike_id,act,reshuffle
0,31000,Eads St & 15th St S,2019-01-02 09:46:15,W23821,-1,0
1,31000,Eads St & 15th St S,2019-01-02 09:58:13,W21229,-1,0
2,31000,Eads St & 15th St S,2019-01-02 12:09:20,W23657,1,0
3,31000,Eads St & 15th St S,2019-01-02 12:12:59,W23657,-1,0
4,31000,Eads St & 15th St S,2019-01-02 17:34:18,W23821,1,0


In [20]:
# Change the "date" column to a datetime format
final_act_df['date'] =  pd.to_datetime(final_act_df['date'], format='%Y-%m-%d %H:%M:%S')

In [21]:
# Sort the dataframe by 'station_id', and 'date'
final_act_df = final_act_df.sort_values(['station_id', 'date'])
final_act_df.tail()

,station_id,station_name,date,bike_id,act,reshuffle
6796829,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0
6796830,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0
6796831,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0
6796832,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0
6796833,32609,W Columbia St & N Washington St,2019-12-30 10:30:27,W23434,-1,0


In [22]:
# Number of unique station names
len(final_act_df['station_id'].unique())

581

## Adding starting bike number for each stations

In [23]:
# Load in the data for 2018 December
bike2018_dec = pd.read_csv("/Users/lee14257/Development/CMU/Perspectives_in Data_Science/Project/2018/201812-capitalbikeshare-tripdata.csv", index_col=None, header=0)

In [24]:
bike2018_dec.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,790,2018-12-01 00:00:44,2018-12-01 00:13:54,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W21477,Member
1,771,2018-12-01 00:00:55,2018-12-01 00:13:46,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W22810,Member
2,775,2018-12-01 00:00:55,2018-12-01 00:13:51,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W21144,Member
3,750,2018-12-01 00:01:08,2018-12-01 00:13:39,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W23321,Member
4,146,2018-12-01 00:02:45,2018-12-01 00:05:11,31124,14th & Irving St NW,31102,11th & Kenyon St NW,W22304,Member


In [25]:
# Repeat preprocessing for 2018
bike2018_dec['Start date'] = pd.to_datetime(bike2018_dec['Start date'], utc=False)
bike2018_dec['End date'] = pd.to_datetime(bike2018_dec['End date'], utc=False)

# Create Availability table for 2018
drop_df2 = bike2018_dec[['Start station number', 'Start station', 'Start date', 'Bike number']].assign(act=-1)
drop_df2.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
pick_df2 = bike2018_dec[['End station number', 'End station', 'End date', 'Bike number']].assign(act=1)
pick_df2.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
act_df2 = pd.concat([drop_df2, pick_df2])
act_df2 = act_df2.assign(reshuffle=0)
act_df2 = act_df2.sort_values(['bike_id', 'date'])

In [26]:
act_df2.head()

,station_id,station_name,date,bike_id,act,reshuffle
137,31227,13th St & New York Ave NW,2018-12-01 01:33:45,23750,-1,0
137,31019,Wilson Blvd & N Edgewood St,2018-12-01 02:50:50,23750,1,0
10853,31039,Wilson Blvd & N Quincy St,2018-12-03 11:32:14,23750,-1,0
10853,31033,Barcroft Community Center,2018-12-03 11:49:20,23750,1,0
7281,31411,Georgia & Missouri Ave NW,2018-12-02 18:39:06,23751,-1,0


In [27]:
# Shift the bike_id column up by 1
act_df2['bike_id_prev'] = act_df2['bike_id'].shift(-1)

In [28]:
# Get last trips for each bike_id (which station they arrived in last)
last_trips = act_df2.loc[(act_df2['bike_id'] != act_df2['bike_id_prev']) & (act_df2['act'] == 1)].drop(columns=['reshuffle', 'bike_id_prev'])

In [29]:
last_trips.head()

,station_id,station_name,date,bike_id,act
10853,31033,Barcroft Community Center,2018-12-03 11:49:20,23750,1
83797,31654,King Greenleaf Rec Center,2018-12-14 13:34:10,23751,1
43922,31628,1st & K St SE,2018-12-08 09:49:14,23757,1
152865,32026,Carroll & Ethan Allen Ave,2018-12-30 12:07:01,51020,1
156503,31096,S Walter Reed Dr & 8th St S,2018-12-31 02:30:34,51033,1


In [30]:
# Sum all the bikes that landed in each station in 2018 December
starting_bike_num = last_trips.groupby('station_id')['act'].agg(['sum'])
starting_bike_num = starting_bike_num.rename(columns={"sum": "act"})

In [31]:
# Assign the date 2019-01-01 as the starting date
starting_bike_num = starting_bike_num.assign(station_name = 'STARTING BIKE NUM', date='2019-01-01 00:00:01', bike_id='starting', reshuffle=0)

In [32]:
# Change 'date' to a datetime type
starting_bike_num = starting_bike_num.reset_index()
starting_bike_num['date'] =  pd.to_datetime(starting_bike_num['date'])
starting_bike_num.head()

,station_id,act,station_name,date,bike_id,reshuffle
0,31000,5,STARTING BIKE NUM,2019-01-01 00:00:01,starting,0
1,31001,4,STARTING BIKE NUM,2019-01-01 00:00:01,starting,0
2,31002,16,STARTING BIKE NUM,2019-01-01 00:00:01,starting,0
3,31003,5,STARTING BIKE NUM,2019-01-01 00:00:01,starting,0
4,31004,7,STARTING BIKE NUM,2019-01-01 00:00:01,starting,0


In [33]:
# Total of 500 stations that have a starting bike number
len(starting_bike_num['station_id'].unique())

500

In [34]:
# Add starting date to final df (concatenate as rows)
final_df_with_startdate = pd.concat([final_act_df, starting_bike_num], ignore_index=True)
final_df_with_startdate['station_id'].replace(0, 31127,inplace=True)

In [35]:
# Sort table by station id and date
final_df_with_startdate = final_df_with_startdate.sort_values(['station_id', 'date'])
final_df_with_startdate.head()

,station_id,station_name,date,bike_id,act,reshuffle
7494220,31000,STARTING BIKE NUM,2019-01-01 00:00:01,starting,5,0
0,31000,Eads St & 15th St S,2019-01-02 09:46:15,W23821,-1,0
1,31000,Eads St & 15th St S,2019-01-02 09:58:13,W21229,-1,0
2,31000,Eads St & 15th St S,2019-01-02 12:09:20,W23657,1,0
3,31000,Eads St & 15th St S,2019-01-02 12:12:59,W23657,-1,0


In [37]:
last_availability = final_df_with_startdate.groupby(['station_id'])['act'].agg(['sum'])
#last_availability.to_csv("last_availability.csv")

In [38]:
# Add availability column through cumsum
final_df_with_startdate = final_df_with_startdate.assign(availability = final_df_with_startdate.groupby('station_id')['act'].transform(pd.Series.cumsum))

In [39]:
# Resulting final table
final_df_with_startdate.head()

,station_id,station_name,date,bike_id,act,reshuffle,availability
7494220,31000,STARTING BIKE NUM,2019-01-01 00:00:01,starting,5,0,5
0,31000,Eads St & 15th St S,2019-01-02 09:46:15,W23821,-1,0,4
1,31000,Eads St & 15th St S,2019-01-02 09:58:13,W21229,-1,0,3
2,31000,Eads St & 15th St S,2019-01-02 12:09:20,W23657,1,0,4
3,31000,Eads St & 15th St S,2019-01-02 12:12:59,W23657,-1,0,3


## Add bike tot_capacity column

In [42]:
# Get total capacity for each station using API
url_station_info = 'https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json'
sta_info = pd.DataFrame(json.loads(requests.get(url_station_info).content)['data']['stations'])
sta_info.head()

,name,station_type,region_id,short_name,rental_uris,capacity,eightd_has_key_dispenser,electric_bike_surcharge_waiver,external_id,has_kiosk,lat,rental_methods,lon,legacy_id,station_id,eightd_station_services
0,Eads St & 15th St S,classic,41,31000,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",15,False,False,082469cc-1f3f-11e7-bf6b-3863bb334450,True,38.858971,"[KEY, CREDITCARD]",-77.053230,1,1,[]
1,Crystal Dr & 20th St S,classic,41,31002,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",17,False,False,08246c35-1f3f-11e7-bf6b-3863bb334450,True,38.856425,"[KEY, CREDITCARD]",-77.049232,3,3,[]
2,Crystal Dr & 15th St S,classic,41,31003,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",16,False,False,08246cd5-1f3f-11e7-bf6b-3863bb334450,True,38.861056,"[KEY, CREDITCARD]",-77.049417,4,4,[]
3,Aurora Hills Cmty Ctr / 18th St & S Hayes St,classic,41,31004,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",12,False,False,08246d68-1f3f-11e7-bf6b-3863bb334450,True,38.857866,"[KEY, CREDITCARD]",-77.059490,5,5,[]
4,Pentagon City Metro / 12th St & S Hayes St,classic,41,31005,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",18,False,False,08246df5-1f3f-11e7-bf6b-3863bb334450,True,38.862303,"[KEY, CREDITCARD]",-77.059936,6,6,[]


In [43]:
capacity = sta_info[['short_name', 'capacity', 'lon', 'lat']]
capacity = capacity.rename(columns={"short_name": "station_id"})
capacity['station_id'] = capacity['station_id'].astype(int)

In [44]:
#.to_csv("bike_capacity_lon_lat.csv", index=False)

In [45]:
capacity.head()

,station_id,capacity,lon,lat
0,31000,15,-77.053230,38.858971
1,31002,17,-77.049232,38.856425
2,31003,16,-77.049417,38.861056
3,31004,12,-77.059490,38.857866
4,31005,18,-77.059936,38.862303


In [46]:
final_act_df_cap = pd.merge(final_df_with_startdate, capacity[['station_id','capacity']],on='station_id', how='left')

In [47]:
final_act_df_cap

,station_id,station_name,date,bike_id,act,reshuffle,availability,capacity
0,31000,STARTING BIKE NUM,2019-01-01 00:00:01,starting,5,0,5,15.0
1,31000,Eads St & 15th St S,2019-01-02 09:46:15,W23821,-1,0,4,15.0
2,31000,Eads St & 15th St S,2019-01-02 09:58:13,W21229,-1,0,3,15.0
3,31000,Eads St & 15th St S,2019-01-02 12:09:20,W23657,1,0,4,15.0
4,31000,Eads St & 15th St S,2019-01-02 12:12:59,W23657,-1,0,3,15.0
...,...,...,...,...,...,...,...,...
7494715,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0,-9,12.0
7494716,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0,-8,12.0
7494717,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0,-9,12.0
7494718,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0,-10,12.0


In [48]:
# Output final table
final_act_df_cap.to_csv("bike2019_transformed.csv", index=False)